<a href="https://colab.research.google.com/github/thiagoscerqueira/fiap_techchallenge_03/blob/main/model_output_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-2y7dv9k3/unsloth_d89558c532d043cda46f060625027e75
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-2y7dv9k3/unsloth_d89558c532d043cda46f060625027e75
  Resolved https://github.com/unslothai/unsloth.git to commit fb77505f8429566f5d21d6ea5318c342e8a67991
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.1 MB/s eta 0:00:00


In [7]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
model_name = "/content/drive/MyDrive/models",
max_seq_length = max_seq_length,
dtype = dtype,
load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.9.post2 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
       

In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

#FastLanguageModel.for_inference(trained_model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Get content for title",
        "Digital IC's and Embedded Core Systems", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Get content for title

### Input:
Digital IC's and Embedded Core Systems

### Response:
"This book is a must-read for anyone interested in the latest trends in embedded systems and microprocessors. It is a great reference for anyone who wants to learn about the latest trends in embedded systems and microprocessors."--Dr. David J. Hunt, Professor of Computer Science, University of California, Santa Barbara"This book is a must-read for anyone interested in the latest trends in embedded systems and microprocessors. It is a great reference for anyone who wants to learn about the latest trends in embedded systems and microprocessors."--Dr. David J. Hunt, Professor of Computer Science, University of California, Santa Barbara</s>


In [9]:
from unsloth import FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True
modelpre, pretokenizerpre = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-chat-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [10]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(modelpre)
inputs = pretokenizerpre(
[
    alpaca_prompt.format(
        "Get content for title",
        "Digital IC's and Embedded Core Systems", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(pretokenizerpre)
_ = modelpre.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Get content for title

### Input:
Digital IC's and Embedded Core Systems

### Response:
1. Digital IC's
2. Embedded Core Systems

3. Digital IC's and Embedded Core Systems

4. Digital IC's and Embedded Core Systems

5. Digital IC's and Embedded Core Systems

6. Digital IC's and Embedded Core Systems

7. Digital IC's and Embedded Core Systems

8. Digital IC's and Embedded Core Systems

9. Digital IC's and Embedded Core Systems

10. Digital IC's and Embedded Core Systems

11. Digital IC's and Embedded Core Systems

12. Digital IC's and Embedded Core Systems

13. Digital IC's and Embedded Core Systems

14. Digital IC's and Embedded Core Systems

15. Digital IC's and Embedded Core Systems

16. Digital IC's and Embedded Core Systems

17. Digital IC's and Embedded Core Systems

18. Digital IC's and Embedded Core Sy